In [257]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load the data
file_path = '/content/drive/MyDrive/reduced_cleaned.csv'
data = pd.read_csv(file_path)

pd.set_option('display.max_columns', None)  # Show all columns

df = pd.read_csv('/content/drive/MyDrive/MovieWithWeatherV3.csv')

columns_to_add = ['title', 'popularity', 'vote_average']
data = pd.merge(data, df[columns_to_add], left_index=True, right_index=True, how='left')

data.drop_duplicates(inplace=True)

data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,keywords_eerie,overview_none,tagline_no,production_artists,production_filmes,cast_Adam Zellen,tagline_none,keywords_survival,keywords_found,overview_her,production_unknown,overview_new,title_the,keywords_joyful,Science Fiction,tagline_is,title_time,keywords_film,Western,keywords_relationship,director_Maxime Bonett,keywords_climate,tagline_us,director_Ryan Ward,keywords_moody,production_warner,keywords_noir,production_production,title_unthinkable,cast_Mike Kelson,keywords_calm,overview_their,Family,keywords_relationships,keywords_live,director_Steven Spielberg,Drama,keywords_fun,overview_in,cast_Auli'i Cravalho,keywords_wistful,production_films,keywords_dramatic,cast_Julie Andrews,overview_his,Unknown,director_Other,cast_Stefan Gota,cast_Sami Lalou,keywords_relaxed,keywords_life,keywords_christmas,History,production_bros,keywords_joyous,keywords_romantic,title_all,title_story,tagline_ll,production_university,director_Quentin Tarantino,cast_Jonathan Urbin,overview_life,keywords_nostalgia,keywords_stop,keywords_bold,keywords_horror,cast_James Marsden,keywords_sentimental,keywords_ghost,overview_is,keywords_surrealism,keywords_complex,tagline_his,keywords_animation,Action,production_pictures,director_Laura Marie Koch,keywords_experimental,title_movie,keywords_war,keywords_warming,cast_Kennedy Williams,keywords_music,production_columbia,keywords_murder,keywords_family,keywords_somber,War,Adventure,production_media,cast_Pierre Sage,keywords_based,keywords_personal,tagline_if,keywords_dreamlike,keywords_hopeful,keywords_gentle,keywords_short,keywords_mountain,keywords_coming,production_studios,keywords_inspirational,keywords_satire,cast_Joel Liss,production_mirri,keywords_creepy,keywords_sports,title_for,keywords_intrigue,overview_to,cast_Joel Hirvonen,production_llc,keywords_suicide,production_cinerain,director_Bas Devos,cast_Rocco Lacivita,keywords_whimsical,keywords_playful,keywords_serene,Animation,keywords_winter,keywords_psychedelic,keywords_philosophical,keywords_dark,keywords_unsettling,cast_Jay Bradley,tagline_it,keywords_new,keywords_lighthearted,keywords_avant,title_adventures,keywords_supernatural,keywords_introspective,Music,overview_story,overview_dark,keywords_contemplative,keywords_complicated,keywords_slow,keywords_strange,tagline_first,keywords_the,keywords_emotional,cast_Carrie-Anne Moss,tagline_for,keywords_biography,production_walt,keywords_wedding,production_apeiron,keywords_environmental,tagline_on,keywords_footage,director_Michael Schultz,cast_Dorkas Kiefer,keywords_lgbt,keywords_foreboding,tagline_you,cast_Ryan Ward,production_cinema,cast_Tobias Kay,production_animation,Documentary,tagline_how,keywords_snow,Comedy,Thriller,keywords_comedy,keywords_rain,title_unravel,cast_Alexandre Lacazette,tagline_when,cast_Kurstin Moser,title_hearts,keywords_night,keywords_demon,cast_Karin Kiurina,keywords_reflective,overview_while,tagline_your,director_Gregor Jordan,tagline_love,keywords_haunting,Crime,keywords_father,keywords_and,title_in,director_Dave Lojek,keywords_mysterious,tagline_of,Horror,Mystery,keywords_intense,keywords_possession,keywords_global,cast_Johnny Depp,keywords_powerful,keywords_paranormal,title_and,tagline_they,tagline_have,tagline_and,keywords_optimistic,director_Samppa Batal,keywords_school,keywords_healing,keywords_surreal,keywords_suspenseful,tagline_the,cast_Lillyana Martinez,director_Taylor D. Adams,production_france,director_Simon Kasto,director_Zack Snyder,keywords_comforting,keywords_american,overview_by,keywords_of,overview_back,TV Movie,keywords_true,cast_Maxime Bonett,cast_Denzel Washington,cast_Dustin Hoffman,cast_Corentin Tolisso,keywords_london,cast_Chiwetel Ejiofor,keywords_love,keywords_meditative,keywords_revenge,keywords_class,tagline_dream,keywords_drama,tagline_he,production_productions,keywords_cinema,keywords_questioning,keywords_sinister,director_Emma Rozanski,director_Rocco Lacivita,keywords_or,cast_Angela Bassett,keywords_nostalgic,keywords_on,keywords_black,keywords_space,t

# Feature Engineering

In [258]:
title = ['title']
weather = ['Clear Sky', 'Few Clouds', 'Scattered Clouds', 'Broken Clouds', 'Shower Rain', 'Rain', 'Thunderstorm', 'Snow', 'Mist']
genre_columns = [
        'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
        'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
        'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Unknown','Western']

In [259]:
# sort the columns based on sum
sorted_columns = data.loc[:, data.columns != 'title'].sum().sort_values(ascending=False).index
data[sorted_columns].sum()

,0
popularity,127440.174
vote_average,47152.214
director_Other,8141.000
Drama,3934.000
tagline_none,3804.000
...,...
cast_Kennedy Williams,4.000
cast_Joel Liss,3.000
cast_Kurstin Moser,3.000
cast_Jay Bradley,3.000


In [261]:
# I select top 50 features ranked by frequency and genres and weathers for modeling,
# over 100 features begins to give extremely unbalanced and ambiguous clusters to map to weathers
# However, current features also contains too much meaningless words like is/are and pronouns but considered as important,
# so the model still rely too much on weather that are already mapped in the cleaned dataset, we might need to reconsider features to include in the training
top_n = 50
features = list(sorted_columns[: top_n])
print('features', features)

# check
print('weather not included' , set(weather) - set(features))
print('genre not included', set(genre_columns) - set(features))

final_features = title + features + list(set(genre_columns) - set(features))
# final_features

features ['popularity', 'vote_average', 'director_Other', 'Drama', 'tagline_none', 'Comedy', 'production_unknown', 'Broken Clouds', 'Horror', 'Thriller', 'Mist', 'title_the', 'Documentary', 'Romance', 'Mystery', 'Snow', 'Thunderstorm', 'Few Clouds', 'Fantasy', 'production_productions', 'Science Fiction', 'overview_to', 'overview_and', 'production_films', 'Rain', 'Crime', 'Animation', 'Action', 'Adventure', 'Scattered Clouds', 'Shower Rain', 'production_pictures', 'Clear Sky', 'overview_in', 'tagline_the', 'keywords_surreal', 'Family', 'overview_his', 'overview_her', 'overview_is', 'Music', 'production_entertainment', 'overview_an', 'keywords_film', 'overview_none', 'production_film', 'overview_for', 'overview_by', 'tagline_of', 'overview_their']
weather not included set()
genre not included {'History', 'Western', 'Unknown', 'TV Movie', 'War'}


# K-Means

In [262]:
data_kmean = data[final_features]
# Column to exclude from duplicate consideration
exclude_column = 'title'

# Drop duplicates ignoring the specified column
data_kmean.drop_duplicates(subset=[col for col in data_kmean.columns if col != exclude_column], inplace=True)

title = data_kmean['title']
X_raw = data_kmean.drop(columns=['title'])

<ipython-input-262-fb94f77545ce>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_kmean.drop_duplicates(subset=[col for col in data_kmean.columns if col != exclude_column], inplace=True)


In [264]:
# the weather data is imbalanced especially for Scattered Clouds and Mist
data_kmean[weather].sum() / len(data)

,0
Clear Sky,0.067752
Few Clouds,0.099224
Scattered Clouds,0.077915
Broken Clouds,0.222926
Shower Rain,0.072779
Rain,0.083597
Thunderstorm,0.100208
Snow,0.102830
Mist,0.172768


In [265]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X_raw)

# Initialize and fit KMeans
kmeans = KMeans(n_clusters=len(weather), random_state=42)
kmeans.fit(X)

# Cluster centers and labels
centers = kmeans.cluster_centers_
labels = kmeans.labels_

# organize the result
df_result = title.to_frame()
df_result['label'] = labels
df_result[X_raw.columns] = X_raw
df_result

,title,label,popularity,vote_average,director_Other,Drama,tagline_none,Comedy,production_unknown,Broken Clouds,Horror,Thriller,Mist,title_the,Documentary,Romance,Mystery,Snow,Thunderstorm,Few Clouds,Fantasy,production_productions,Science Fiction,overview_to,overview_and,production_films,Rain,Crime,Animation,Action,Adventure,Scattered Clouds,Shower Rain,production_pictures,Clear Sky,overview_in,tagline_the,keywords_surreal,Family,overview_his,overview_her,overview_is,Music,production_entertainment,overview_an,keywords_film,overview_none,production_film,overview_for,overview_by,tagline_of,overview_their,History,Western,Unknown,TV Movie,War
0,Radio,3,26.728,7.2,1.0,1,0.0,0,0.0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.256097,0,0.179944,0.174950,0.000000,0,0,0,0,0,0,0,0.259308,1,0.203392,0.000000,0.0,0,0.174797,0.000000,0.247866,0,0.000000,0.000000,0.000000,0.0,0.0,0.099218,0.000000,0.0,0.000000,0,0,0,0,0
1,Porch Pirates,3,6.683,0.0,0.0,1,0.0,1,0.0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0,0.094018,0.137113,0.325244,0,0,0,0,0,0,0,0.000000,1,0.053134,0.129411,0.0,1,0.205489,0.000000,0.064753,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0,0,0,0,0
2,On the Sly: In Search of the Family Stone,3,3.304,0.0,1.0,0,1.0,0,1.0,0,0,0,0,0.405769,1,0,0,0,0,0,0,0.000000,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.000000,1,0.000000,0.000000,0.0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.0,0.290771,0.000000,0.0,0.000000,0,0,0,0,0
3,Chasing Ghosts,3,2.853,6.4,0.0,1,0.0,1,0.0,0,0,0,0,0.000000,0,0,0,0,0,0,0,0.000000,0,0.169203,0.054836,0.379438,0,0,0,0,0,0,0,0.000000,1,0.127501,0.000000,0.0,1,0.164363,0.000000,0.077691,0,0.000000,0.087383,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0,0,0,0,0
4,Clocking The T,3,2.662,0.0,1.0,1,0.0,1,0.0,0,0,0,0,1.000000,0,1,0,0,0,0,0,0.000000,0,0.139171,0.067655,0.000000,0,0,0,0,0,0,0,0.000000,1,0.157307,0.000000,0.0,0,0.000000,0.000000,0.095852,0,0.000000,0.000000,0.206858,0.0,0.0,0.115105,0.000000,0.0,0.126118,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10052,The Shadow People,2,5.558,4.6,1.0,0,0.0,0,0.0,0,0,1,1,0.218121,0,0,0,0,0,0,0,0.000000,0,0.098785,0.192087,1.000000,0,0,0,0,0,0,0,0.000000,0,0.000000,0.313913,0.0,0,0.071969,0.168534,0.136073,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.268557,0,0,0,0,0
10053,The Hat Man: Documented Cases of Pure Evil,6,4.945,6.3,1.0,0,1.0,0,1.0,0,0,0,1,0.241262,1,0,0,0,0,0,0,0.000000,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.000000,0,0.000000,0.000000,0.0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0,0,0,0,0
10054,A Febre,6,3.744,0.0,1.0,0,0.0,0,0.0,0,1,0,1,0.000000,0,0,0,0,0,0,1,0.000000,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.000000,0,0.108012,0.000000,0.0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.167902,0.0,0.0,0.158071,0.000000,0.0,0.000000,0,0,0,0,0
10055,The Things With The Glowing Green Eyes,2,1.201,0.0,1.0,0,0.0,1,0.0,0,1,0,1,0.320523,0,0,0,0,0,0,0,1.000000,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0.000000,0,0.000000,0.000000,0.0,0,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.357790,0,0,0,0,0


In [285]:
df_result.to_csv('/content/drive/MyDrive/Clustered_Data.csv', index=False)

# Try Recommendation

In [286]:
df = pd.read_csv('/content/drive/MyDrive/Clustered_Data.csv')

# Weather-to-Cluster Mapping (9 Clusters)
weather_to_cluster = {
    "Clear Sky": [3],
    "Few Clouds": [4],
    "Scattered Clouds": [0],
    "Broken Clouds": [2],
    "Shower Rain": [7],
    "Rain": [5],
    "Thunderstorm": [8],
    "Snow": [1],
    "Mist": [6]
}


def recommend_movies(weather, top_n=5):
    # Get clusters for the given weather
    clusters = weather_to_cluster.get(weather, [])

    # Filter movies by clusters
    recommendations = df_result[df["label"].isin(clusters)]

    # Sort by vote_average and popularity
    recommendations = recommendations.sort_values(by=["vote_average", "popularity"], ascending=False)

    # Genre columns
    genre_columns = [
        'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
        'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery',
        'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'
    ]

    # Add a new column with genres as a list
    recommendations['genres'] = recommendations[genre_columns].apply(
        lambda row: [genre for genre in genre_columns if row[genre] == 1], axis=1
    )

    # Select relevant columns
    relevant_columns = ['title', 'vote_average', 'popularity', 'genres']
    return recommendations[relevant_columns].head(top_n)

In [287]:
# Example Recommendation for "Clear Sky" (Sunny)
weather = "Clear Sky"
recommendations = recommend_movies(weather)
print(f"Recommendations for '{weather}':")
print(recommendations)

Recommendations for 'Clear Sky':
                                        title  vote_average  popularity  \
506                   LOOK! The Invisible Man          10.0       6.982   
499  Occurrences of Questionable Significance          10.0       6.419   
67                              Fox & Penguin          10.0       5.568   
382                    Krishnam Pranaya Sakhi          10.0       5.356   
328                            Paradise Beach          10.0       4.194   

                                     genres  
506               [Comedy, Science Fiction]  
499       [Comedy, Fantasy, Music, Romance]  
67   [Comedy, Documentary, Family, Romance]  
382                       [Comedy, Romance]  
328                       [Comedy, Fantasy]  
